# LDA+DNN

## 1.导入工具包

In [44]:
# import os
import csv
import time
import datetime
import random
import json
from collections import Counter
from math import sqrt
import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed,Input,Conv2D,MaxPool2D,concatenate,Flatten,Dense,Dropout,Embedding,SimpleRNN,Reshape,GRU,LSTM
from tensorflow.keras import Sequential,optimizers,losses
from tensorflow.keras.models import Model,model_from_yaml
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from bs4 import BeautifulSoup
import logging
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import multiprocessing
import yaml
import jieba
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from sklearn.model_selection import train_test_split

## 2.设置参数

In [45]:
vocab_dim = 60
maxlen = 50
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
batch_size = 64
n_epoch = 10
input_length = 60
cpu_count = multiprocessing.cpu_count()

## 3.读取数据集

In [46]:
neg=pd.read_excel('../../data/day10-nlp-data/film_review/neg.xlsx',index=None,header = None)
pos=pd.read_excel('../../data/day10-nlp-data/film_review/pos.xlsx',index=None,header = None)

## 4. 查看数据集

### 消极评论

In [47]:
neg.head()

,0,1
0,期望太大，失望也就大了,0
1,没有太强的代入感，感觉看电影没那么激动，或者是每个前段过短？,0
2,**片，拍的真心不咋地,0
3,响应华诞庆生,0
4,太浪费时间了,0


### 积极评论

In [48]:
pos.head()

,0,1
0,很不错的影片，敬礼祖国，敬礼高,1
1,很喜欢，很感动,1
2,我爱我的祖国，很震撼,1
3,第一段看了有击中泪点的地方，还有女排那集也很好，其他的几集觉得差点差点没有到点。不过还是很值...,1
4,合唱我的祖国,1


## 5.数据连接

In [49]:
comment=np.concatenate((pos[0], neg[0]))
y = np.concatenate((np.ones(len(pos),dtype=int), np.zeros(len(neg),dtype=int)))
comment_new = comment.astype(str).tolist()

### 6.分词预处理

In [50]:
def get_stopword_list():
    """
    建立停用词表
    """
    stop_word_path = '../../data/day10-nlp-data/chineseStopWords.txt'
    stopword_list = [sw.replace('\n', '').replace(' ', '') for sw in open(stop_word_path, encoding='gb18030').readlines()]
    return stopword_list
stopword_list = get_stopword_list()

In [51]:
stopword_list.append('@')
stopword_list.append(' ')
stopword_list.append('')
stopword_list.append('..........................................................................................')
stopword_list.append('\n')
stopword_list.append('$')

def cut_remove(text):
    """
    分词
    """
    news_list = []
    for str_text in text:
        str_words = ','.join(jieba.cut(str_text)).split(',')
        for word in str_words:
            if word not in stopword_list:
                news_list.append(word)
    return news_list

In [52]:
comment_new = cut_remove(comment_new)

## 7.LDA主题模型

In [53]:
"""
1.加载gensim，建立lda模型
"""
dictionary = Dictionary([comment_new])
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in [comment_new]]

"""
2.使用LDA算法求得前5的topic，同时生成的topic在之后也会被使用到来定义文本所属主题  
"""
 
NUM_TOPICS = 5
ldamodel = LdaModel(corpus,num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

"""
3.保存lda模型
"""
ldamodel.save('model/LDA/model5.gensim')


topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

Dictionary(11247 unique tokens: ['诚意', '那辆车', '司机', '滚出去', '赶到']...)
(0, '0.033*"好看" + 0.029*"祖国" + 0.018*"故事" + 0.017*"好" + 0.015*"中国"')
(1, '0.001*"好看" + 0.001*"祖国" + 0.001*"故事" + 0.001*"好" + 0.000*"中国"')
(2, '0.000*"祖国" + 0.000*"好看" + 0.000*"故事" + 0.000*"好" + 0.000*"感动"')
(3, '0.000*"祖国" + 0.000*"好看" + 0.000*"故事" + 0.000*"好" + 0.000*"中国"')
(4, '0.001*"好看" + 0.001*"祖国" + 0.000*"好" + 0.000*"中国" + 0.000*"故事"')


In [54]:
comment

array(['很不错的影片，敬礼祖国，敬礼高', '很喜欢，很感动', '我爱我的祖国，很震撼', ...,
       '此片我为祖国打10分，为导演、编辑打负5分。首先整部片子杂乱无章，虽然是有插叙，有倒序手法，但很多细节交代不清，大部分都是带过，影片还有很多大事记没有表现，一是2003年众志成城抗击非典，2008年汶川地震，宇航员出仓，天宫对接等等，这些影片都没有表现。  导演的水平参差不齐，本以为最震撼的是开国大典，虽然有气派，有感情，表现了普通人对开国大典的期待、关注。但最后却是缺少欢呼，最起码应该给些大家喊共产党万岁，毛**万岁这样的镜头，带动一下气氛吧。让大家感受一下开国大典人民的热情总归好吧。电影很多细节交代都不清晰。导演和编辑是糊弄洋鬼子吗？',
       '洗腦愛國片嗯嗯', '除了第一段还好。剩下拍的什么东西！特别是倒数第二段。'], dtype=object)

In [55]:
def lda2list(lda,topic_n):
    lda_dict = dict(lda)
    lda_list = [0] * topic_n
    for i in range(topic_n):
        lda_list[i] = lda_dict.get(i,0)
    return lda_list

comment = comment.astype(str).tolist()
comment = [','.join(jieba.cut(document)).split(',') for document in comment]


def remove_stopwords(text):
    """
    去停用词
    """
    for index in text[:]:
        if index in stopword_list:
            text.remove(index)
    return text       
comment = [remove_stopwords(document) for document in comment]    

# 句子主题向量化
comment_vec =[]
for comment_content in comment:
    comment_bow = dictionary.doc2bow(comment_content)     
    comment_lda = lda2list(ldamodel[comment_bow],6) 
    comment_vec.append(comment_lda)

In [56]:
X = np.array(comment_vec)

## 8.加载词向量模型

In [57]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [58]:
x_train

array([[0.73325688, 0.0667034 , 0.06667776, 0.06667524, 0.06668678,
        0.        ],
       [0.79958785, 0.05010617, 0.0500997 , 0.05010005, 0.05010621,
        0.        ],
       [0.83994818, 0.04002053, 0.04000643, 0.04000465, 0.04002024,
        0.        ],
       ...,
       [0.92723513, 0.01819283, 0.01818999, 0.01818982, 0.0181923 ,
        0.        ],
       [0.88548046, 0.02863171, 0.02862848, 0.02862854, 0.02863077,
        0.        ],
       [0.88488394, 0.0287798 , 0.02877794, 0.02877818, 0.02878014,
        0.        ]])

## 9.创建模型

In [59]:
"""
1.初始化：
    glorot_uniform均匀分布初始化
    优化函数Adam
"""

init = K.initializers.glorot_uniform(seed=2)
simple_adam = K.optimizers.Adam()

"""
2.定义DNN模型：
    搭建的DNN由输入层、隐藏层、输出层和sigmoid函数组成，其中输入层由训练集的维度决定，隐藏层有一层，
    包括32个神经元，之后就是输出层，由1个神经元组成，最后，sigmoid，用于解决二分类问题而创建。
"""
model = K.models.Sequential()
model.add(K.layers.Dense(units=32, input_dim=x_train.shape[1], kernel_initializer=init, activation='sigmoid'))
model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=simple_adam, metrics=['accuracy'])

"""
3.训练模型fit
"""
h = model.fit(X_new_train, y_train, batch_size=100, epochs=5, shuffle=True, verbose=1)

"""
4.检验模型evaluate
"""
eval = model.evaluate(X_test, y_test, verbose=0)
print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" \
          % (eval[0], eval[1] * 100))

Starting training
Train on 8156 samples
Epoch 1/5
8156/8156 [==============================] - 2s 280us/sample - loss: 0.6929 - accuracy: 0.5186